In [2]:
import pandas as pd
import numpy as np 
import pickle 
import requests
import time
from tqdm import tqdm

import vk_api

In [11]:
import networkx as nx
from scipy.sparse import csr_matrix, lil_matrix
from communities.algorithms import louvain_method, hierarchical_clustering

In [5]:
# Функция для записи строки в файл формата pickle
def write_string_to_pickle_file(string, file_path):
    """
    Записывает строку в файл формата pickle.
    
    Аргументы:
        string (str): Строка, которую нужно записать в файл.
        file_path (str): Путь к файлу, в который нужно записать строку.
    """
    with open(file_path, 'wb') as f:
        pickle.dump(string, f)

# Функция для чтения строки из файла формата pickle
def read_string_from_pickle_file(file_path):
    """
    Читает строку из файла формата pickle.
    
    Аргументы:
        file_path (str): Путь к файлу, из которого нужно прочитать строку.
        
    Возвращает:
        str: Прочитанная из файла строка.
    """
    with open(file_path, 'rb') as f:
        string = pickle.load(f)
    return string

In [7]:
# Читаем строку с токеном из файла формата pickle
access_token = read_string_from_pickle_file("vk_token_string.pkl")

# Создаем сессию API ВКонтакте с использованием токена
vkApiSession= vk_api.VkApi(token=access_token)

# Получаем объект API пользователя
vkU = vkApiSession.get_api()

In [8]:
# Функция для получения списка id всех друзей пользователя
def get_all_friend_ids(user_id, access_token):
    friend_ids = [] # список для хранения id друзей
    offset = 0 # смещение для получения следующей порции данных
    
    # Цикл для получения всех друзей пользователя
    while True:
        response = requests.get('https://api.vk.com/method/friends.get', params={
            'user_id': user_id,
            'access_token': access_token,
            'count': 1000, # количество друзей, запрашиваемых за один запрос
            'offset': offset, # смещение для получения следующей порции данных
            'fields': 'id', # запрашиваем только id друзей
            'v': '5.131' # версия API ВКонтакте
        }).json()
        
        # Если ответ содержит поле 'response', то добавляем id друзей в список friend_ids
        if 'response' in response:
            items = response['response']['items']
            if len(items) == 0:
                break # если список друзей пуст, то выходим из цикла
            
            for item in items:
                friend_ids.append(item['id'])
            
            offset += 1000 # увеличиваем смещение для получения следующей порции данных
        else:
            print(response['error']['error_msg']) # выводим сообщение об ошибке
            break
    
    return friend_ids

In [9]:
# Функция для получения всех друзей пользователя
# n_iterations - количество итераций
# delay - задержка между запросами
# initial_id - идентификатор пользователя, для которого нужно получить всех друзей

def get_all_friends(n_iterations, delay, initial_id):

    # Создаем множество идентификаторов для парсинга, добавляем начальный идентификатор
    set_of_ids_to_parse = set()
    set_of_ids_to_parse.add(initial_id)

    # Создаем список DataFrame'ов
    list_of_dfs = []

    # Проходимся по всем итерациям
    for i in range(n_iterations):
        
        counter = 0

        # Проходимся по всем идентификаторам в множестве для парсинга
        for id in tqdm(set_of_ids_to_parse):

            # Задержка между запросами
            time.sleep(delay)

            # Получаем список идентификаторов друзей пользователя и создаем DataFrame с парами (идентификатор пользователя, идентификатор друга)
            list_of_friends = get_all_friend_ids(id, access_token)
            list_of_dfs.append(pd.DataFrame(list(zip([id] * len(list_of_friends),  list_of_friends))))
            
            counter += 1
            
            # Каждые 1000 итераций сохраняем DataFrame в файл
            if counter % 1000 == 0:
                combined_df = pd.concat(list_of_dfs, axis=0)
                combined_df.to_pickle(f"combined_df_iteration{i}_{counter}.pkl")
                
        # Объединяем все DataFrame'ы в один и сохраняем его в файл
        combined_df = pd.concat(list_of_dfs, axis=0)
        combined_df.to_pickle(f"combined_df_iteration{i}_full.pkl")

        # Обновляем множество для парсинга - добавляем всех друзей и удаляем уже обработанных пользователей
        set_of_ids_to_parse = set()
        set_of_ids_to_parse.update(combined_df.iloc[:,1])
        set_of_ids_to_parse.difference_update(combined_df.iloc[:,0])

    # Объединяем все DataFrame'ы в один и изменяем названия столбцов
    combined_df_final = pd.concat(list_of_dfs, axis=0)
    combined_df_final.columns = ["from", "to"]

    return combined_df_final


In [10]:
initial_id = 124150763
N_ITERATIONS = 2
DELAY = 1

get_all_friends(N_ITERATIONS, DELAY, initial_id)

 11%|█         | 7/65 [00:11<01:27,  1.51s/it]

This profile is private


 12%|█▏        | 8/65 [00:12<01:21,  1.43s/it]

User was deleted or banned


 17%|█▋        | 11/65 [00:17<01:20,  1.48s/it]

...

 98%|█████████▊| 64/65 [01:40<00:01,  1.50s/it]

This profile is private


100%|██████████| 65/65 [01:41<00:00,  1.56s/it]

This profile is private


,from,to
0,124150763,682502
1,124150763,1518757
2,124150763,6099012
3,124150763,9389942
4,124150763,9681259
...,...,...
287,73311606,674428820
288,73311606,694150279
289,73311606,706174451
290,73311606,740539286


In [14]:
with open("combined_df_iteration1_full.pkl", 'rb') as f:
    edge_list = pickle.load(f)

In [16]:
# Функция для преобразования списка в словарь
# lst - список, который нужно преобразовать в словарь

def list_to_dict(lst):
    d = {}
    # Проходимся по всем элементам списка
    for i in range(len(lst)):
        # Добавляем элемент в словарь с ключом, равным строковому представлению элемента, и значением, равным индексу элемента в списке
        d[str(lst[i])] = i
    return d

In [17]:
# Функция для получения матрицы смежности из списка ребер
# edge_list - список ребер графа в формате [(from_node, to_node), ...]

def get_adj_matrix(edge_list):

    # Получаем уникальные узлы из списка ребер и сортируем их
    nodes = sorted(list(set(edge_list.iloc[:,0]).union(set(edge_list.iloc[:,1]))))
    num_nodes = len(nodes)

    # Преобразуем список узлов в словарь, где ключи - строковые представления узлов, а значения - их индексы в списке узлов
    ids_to_indeces = list_to_dict(nodes)

    # Создаем разреженную матрицу с размером num_nodes x num_nodes и типом данных bool
    adjacency_matrix = lil_matrix((num_nodes, num_nodes), dtype=bool)

    # Проходимся по всем ребрам графа
    for t in tqdm(edge_list.itertuples()):
        from_node = t._1
        to_node = t._2
        from_node_index = ids_to_indeces[str(from_node)]
        to_node_index = ids_to_indeces[str(to_node)]
        # Устанавливаем значение True в ячейках матрицы, соответствующих соответствующим узлам
        adjacency_matrix[from_node_index, to_node_index] = True
        adjacency_matrix[to_node_index, from_node_index] = True
            
    return adjacency_matrix.toarray().astype(np.int8), ids_to_indeces, num_nodes, nodes


In [19]:
adj_matrix_int, ids_to_indeces, num_nodes, nodes = get_adj_matrix(edge_list)

10595it [00:00, 28252.90it/s]


In [20]:
res = hierarchical_clustering(adj_matrix_int, linkage = "complete", metric = "cosine")

In [ ]:
list(filter(lambda x: len(x) > 1, res))

In [27]:
#write_string_to_pickle_file(res, "res.pkl")

Как можно видеть, алгоритм иерархической кластеризации обнаружил только одно сообщество с более чем 1-й вершиной. Следовательно, задачу обнаружения сообществ в графе можно считать невыполненной. Если бы кластеры получилось выделить, дальнейший проект подразумевал следующие стадии: 
1. Визуализацию полученных сообществ 
2. Расчет модулярности графа 
3. Создание приложения, включающего скачивания данных, выделение сообществ и визуализации 


Относительно причин, почему не получилось выделить сообщества - можно предположить, что выбранный алгоритм не подходит для работы в этим типом графов (крайне разряженный, степень плотности около 0.004%). Впрочем, другие алгоритмы или подразумевали hard clustering, что также не подходит к задаче выделения сообществ, или работали также плохо.